In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

In [2]:
# Read the csv file into a pandas DataFrame

diabetes = pd.read_csv('diabetes_frankfurt.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2,138,62,35,0,33.6,0.127,47,1
1,0,84,82,31,125,38.2,0.233,23,0
2,0,145,0,0,0,44.2,0.630,31,1
3,0,135,68,42,250,42.3,0.365,24,1
4,1,139,62,41,480,40.7,0.536,21,0


In [3]:
#data2 = df[["Age"]] 
data2 = diabetes.drop(columns=["Outcome"])
X = data2
#X = data
y = diabetes["Outcome"]
print(X.shape, y.shape)

(2000, 8) (2000,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [5]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [7]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=8))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               900       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 11,202
Trainable params: 11,202
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [10]:
callbacks = [EarlyStopping(monitor='loss', patience=2)]
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
47/47 - 0s - loss: 0.6447 - accuracy: 0.6420
Epoch 2/60
47/47 - 0s - loss: 0.5860 - accuracy: 0.6840
Epoch 3/60
47/47 - 0s - loss: 0.5434 - accuracy: 0.7193
Epoch 4/60
47/47 - 0s - loss: 0.5087 - accuracy: 0.7540
Epoch 5/60
47/47 - 0s - loss: 0.4796 - accuracy: 0.7833
Epoch 6/60
47/47 - 0s - loss: 0.4664 - accuracy: 0.7760
Epoch 7/60
47/47 - 0s - loss: 0.4588 - accuracy: 0.7927
Epoch 8/60
47/47 - 0s - loss: 0.4432 - accuracy: 0.7907
Epoch 9/60
47/47 - 0s - loss: 0.4418 - accuracy: 0.7940
Epoch 10/60
47/47 - 0s - loss: 0.4319 - accuracy: 0.7980
Epoch 11/60
47/47 - 0s - loss: 0.4347 - accuracy: 0.8027
Epoch 12/60
47/47 - 0s - loss: 0.4310 - accuracy: 0.8027
Epoch 13/60
47/47 - 0s - loss: 0.4305 - accuracy: 0.8007
Epoch 14/60
47/47 - 0s - loss: 0.4257 - accuracy: 0.8047
Epoch 15/60
47/47 - 0s - loss: 0.4206 - accuracy: 0.8100
Epoch 16/60
47/47 - 0s - loss: 0.4153 - accuracy: 0.8133
Epoch 17/60
47/47 - 0s - loss: 0.4139 - accuracy: 0.8080
Epoch 18/60
47/47 - 0s - loss: 0.4135 - 

In [11]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, 
    y_test_categorical)

print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

16/16 [==============================] - 0s 1ms/step - loss: 0.4892 - accuracy: 0.7480
Normal Neural Network - Loss: 0.48923489451408386, Accuracy: 0.7480000257492065


In [12]:
TensorTest = model.predict_classes(X_test_scaled)
TensorTest = label_encoder.inverse_transform(TensorTest)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [13]:
print(TensorTest[:5])
print(y_test[:5])

[0 1 0 0 1]
674     0
1699    1
1282    1
1315    0
1210    1
Name: Outcome, dtype: int64


In [14]:
y_index = list(y_test.index)
df = pd.DataFrame(model.predict(X_test_scaled)).rename(columns={0:"Diabetes%",1:"NonDiabetes%"})
df["y_index"]=y_index


In [16]:
#result = pd.DataFrame({"Diabetes%": TensorTest[0], "NonDiabetes%": TensorTest[0]})

In [17]:
df.to_csv('diabetes_TensorTest.csv',index=False)

In [18]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [19]:
prediction_labels 

array([0, 1, 0, 0, 1])